# Connection rules

## Introduction

This notebook implements the connection rules that fix the cases where the sequence of activation-deactivation is not normal.
The rules are as follows:
<img src="files/rules.png">

The only cases that might resulting in ADDING datapoints to the connectivity log are:

online X -> online Y, online X -> offline Y

because it might be necessary to add a point if the server shut down before the second event

All other cases are either normal or can be easily taken care of in the function that determines the device activation (See [next notebook](05_Connected_Devices_Extraction.ipynb))

## Connectivity events fix

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import aux_balena as aux
import importlib
from datetime import datetime, timedelta
%matplotlib inline

In [3]:
dflog = aux.read_log('connectivity_events_clean02.csv')

Mem. usage decreased to 142.03 Mb (51.5% reduction)
Log read in 15.2 s
Converting to timestamp field:  timestamp
Done!


In [4]:
dflog.head()

timestamp  device_id  user_id  server_id  connected
0 2015-05-31 14:30:00          1        1          2       True
1 2015-06-02 10:30:00          1        1          2      False
2 2015-05-31 14:30:00          2        2          2       True
3 2015-06-05 20:30:00          2        2          2      False
4 2015-05-31 14:30:00          3        1          2       True

In [7]:
dflog_clean = dflog.copy()

The following cell reports at each datapoint the difference between that point and the following for device_id and server_id. This will be used to segment the dataset by users and servers

In [8]:
dflog_clean['device_diff'] = -dflog_clean.device_id.diff(periods=-1)
dflog_clean['server_diff'] = -dflog_clean.server_id.diff(periods=-1)

To implement the connection rules, it is necessary to have for each datapoint the connected status and the timestamp of the point immediately following

In [11]:
next_connected = dflog_clean['connected'].iloc[1:]
next_connected=next_connected.append(pd.Series([False]))

In [12]:
next_timestamp = dflog_clean['timestamp'].iloc[1:]
next_timestamp = next_timestamp.append(pd.Series([dflog_clean['timestamp'].iloc[-1] ]))

In [13]:
# append to dataframe
dflog_clean['next_connected']=next_connected.values
dflog_clean['next_timestamp']=next_timestamp.values

In [15]:
dflog_clean.head(10)

timestamp  device_id  user_id  server_id  connected  device_diff  \
0 2015-05-31 14:30:00          1        1          2       True         -0.0   
1 2015-06-02 10:30:00          1        1          2      False          1.0   
2 2015-05-31 14:30:00          2        2          2       True         -0.0   
3 2015-06-05 20:30:00          2        2          2      False          1.0   
4 2015-05-31 14:30:00          3        1          2       True         -0.0   
5 2015-06-03 05:30:00          3        1          2      False         -0.0   
6 2015-06-03 06:30:00          3        1          2       True         -0.0   
7 2015-06-19 20:30:00          3        1          2      False          1.0   
8 2015-05-31 14:30:00          4        1          2       True         -0.0   
9 2015-06-01 16:30:00          4        1          2      False          1.0   

   server_diff  next_connected      next_timestamp  
0         -0.0           False 2015-06-02 10:30:00  
1         -0.0            True 2015-05-31 14:30:00  
2         -0.0           False 2015-06-05 20:30:00  
3         -0.0            True 2015-05-31 14:30:00  
4         -0.0           False 2015-06-03 05:30:00  
5         -0.0            True 2015-06-03 06:30:00  
6         -0.0           False 2015-06-19 20:30:00  
7         -0.0            True 2015-05-31 14:30:00  
8         -0.0           False 2015-06-01 16:30:00  
9         -0.0            True 2015-05-31 14:30:00

### Case: ON X -> OFF Y

In [16]:
dx_on_off = dflog_clean[(dflog_clean.device_diff==0) & (dflog_clean.server_diff!=0)\
                        & (dflog_clean.connected==True) & (dflog_clean.next_connected==False)]
dx_on_off.shape

(0, 9)

There is no case ON X -> ON Y

### Case ON X -> ON Y

In [17]:
dx_on_on = dflog_clean[(dflog_clean.device_diff==0) & (dflog_clean.server_diff!=0)\
                        & (dflog_clean.connected==True) & (dflog_clean.next_connected==True)]
dx_on_on.shape


(78522, 9)

There are 78522 cases of ON X -> ON Y

In [18]:
dx_on_on = dx_on_on.copy()

**Read in server log**

In [37]:
dfserver = pd.read_csv('servers_clean01.csv',parse_dates=[1,2],infer_datetime_format=True,index_col='server_id')

In [38]:
dfserver.dtypes

created_at      datetime64[ns]
destroyed_at    datetime64[ns]
dtype: object

In [39]:
dfserver.head()

created_at            destroyed_at
server_id                                                   
2         2015-05-31 14:30:00.000000 2017-08-03 22:42:57.929
3         2017-08-03 22:43:07.392988 2017-08-03 22:46:50.279
4         2017-08-03 22:46:57.243687 2017-08-08 01:02:16.079
5         2017-08-08 01:02:26.761809 2017-08-29 19:57:59.039
6         2017-08-29 19:58:06.928351 2017-08-29 20:10:36.166

Take server destruction time from servers

In [40]:
dx_on_on['destroyed_at'] = dfserver.loc[dx_on_on.server_id]['destroyed_at'].values

In [41]:
dx_on_on.head()

timestamp  device_id  user_id  server_id  connected  \
1730 2017-08-03 22:34:33.055         23       11          2       True   
1731 2017-08-03 22:44:47.252         23       11          3       True   
2937 2017-08-08 00:54:10.334         23       11          4       True   
9294 2017-08-29 19:53:54.485         23       11          5       True   
9297 2017-08-29 20:04:48.038         23       11          6       True   

      device_diff  server_diff  next_connected          next_timestamp  \
1730         -0.0          1.0            True 2017-08-03 22:44:47.252   
1731         -0.0          1.0            True 2017-08-03 22:47:21.922   
2937         -0.0          1.0            True 2017-08-08 01:03:02.062   
9294         -0.0          1.0            True 2017-08-29 20:03:53.248   
9297         -0.0          2.0            True 2017-08-29 20:14:05.879   

                destroyed_at  
1730 2017-08-03 22:42:57.929  
1731 2017-08-03 22:46:50.279  
2937 2017-08-08 01:02:16.079  
9294 2017-08-29 19:57:59.039  
9297 2017-08-29 20:10:36.166

### Destruction occurring AFTER device connected to new server [NO ACTION]

In [42]:
sum(dx_on_on.destroyed_at > dx_on_on.next_timestamp)

18

ON X -> ON Y occurs 18 times. No action required

### Destruction occurring BEFORE device connected to new server - [Add point]

In [43]:
sum(dx_on_on.destroyed_at < dx_on_on.next_timestamp)

78504

In [44]:
# destruction occurring BEFORE device connected to new server
dx_on_on_before = dx_on_on.copy()
dx_on_on_before = dx_on_on_before[dx_on_on_before.destroyed_at < dx_on_on_before.next_timestamp]
dx_on_on_before.head(6)

timestamp  device_id  user_id  server_id  connected  \
1730  2017-08-03 22:34:33.055         23       11          2       True   
1731  2017-08-03 22:44:47.252         23       11          3       True   
2937  2017-08-08 00:54:10.334         23       11          4       True   
9294  2017-08-29 19:53:54.485         23       11          5       True   
9297  2017-08-29 20:04:48.038         23       11          6       True   
10344 2017-09-02 11:35:22.494         23       11          8       True   

       device_diff  server_diff  next_connected          next_timestamp  \
1730          -0.0          1.0            True 2017-08-03 22:44:47.252   
1731          -0.0          1.0            True 2017-08-03 22:47:21.922   
2937          -0.0          1.0            True 2017-08-08 01:03:02.062   
9294          -0.0          1.0            True 2017-08-29 20:03:53.248   
9297          -0.0          2.0            True 2017-08-29 20:14:05.879   
10344         -0.0          1.0            True 2017-09-02 12:37:47.120   

                 destroyed_at  
1730  2017-08-03 22:42:57.929  
1731  2017-08-03 22:46:50.279  
2937  2017-08-08 01:02:16.079  
9294  2017-08-29 19:57:59.039  
9297  2017-08-29 20:10:36.166  
10344 2017-09-02 12:32:37.348

Create df based on dx_on_on_before, where timestamp is replaced with destroyed_at and connected is set to False

In [45]:
df_to_add = dx_on_on_before.copy()
df_to_add = df_to_add[['destroyed_at','device_id','user_id','server_id','connected']]

df_to_add.head()

destroyed_at  device_id  user_id  server_id  connected
1730 2017-08-03 22:42:57.929         23       11          2       True
1731 2017-08-03 22:46:50.279         23       11          3       True
2937 2017-08-08 01:02:16.079         23       11          4       True
9294 2017-08-29 19:57:59.039         23       11          5       True
9297 2017-08-29 20:10:36.166         23       11          6       True

In [46]:
df_to_add.rename(columns={'destroyed_at': 'timestamp'}, inplace=True)
# set connected to false (as server has just been destroyed)
df_to_add.connected = False
df_to_add.head()

timestamp  device_id  user_id  server_id  connected
1730 2017-08-03 22:42:57.929         23       11          2      False
1731 2017-08-03 22:46:50.279         23       11          3      False
2937 2017-08-08 01:02:16.079         23       11          4      False
9294 2017-08-29 19:57:59.039         23       11          5      False
9297 2017-08-29 20:10:36.166         23       11          6      False

Create new log dataframe by concatenating new datapoints to previous dataframe

In [47]:
dflog_comp = dflog_clean.copy()
dflog_comp = dflog_comp[['timestamp','device_id','user_id','server_id','connected']]
dflog_comp = pd.concat([dflog_comp, df_to_add], ignore_index = True)
dflog_comp.head()

timestamp  device_id  user_id  server_id  connected
0 2015-05-31 14:30:00          1        1          2       True
1 2015-06-02 10:30:00          1        1          2      False
2 2015-05-31 14:30:00          2        2          2       True
3 2015-06-05 20:30:00          2        2          2      False
4 2015-05-31 14:30:00          3        1          2       True

Order by device and timestamp

In [48]:
dflog_comp.sort_values(by=['device_id','timestamp'],inplace=True)
dflog_comp.head()

timestamp  device_id  user_id  server_id  connected
0 2015-05-31 14:30:00          1        1          2       True
1 2015-06-02 10:30:00          1        1          2      False
2 2015-05-31 14:30:00          2        2          2       True
3 2015-06-05 20:30:00          2        2          2      False
4 2015-05-31 14:30:00          3        1          2       True

In [52]:
len(dflog_comp) - len(dflog)

78504

Create csv file

In [53]:
dflog_comp.to_csv('connectivity_events_clean_Final.csv',index=False)